# OpenStreetMap Data Wrangling Project

## 1. Introdução

O OpenStreetMap (OSM) é um projeto *open source* que procura criar um mapa gratuito do mundo inteiro a partir de dados inseridos voluntariamente. É um esforço colaborativo com mais de 2 milhões de contribuidores. Os dados do OpenStreetMap estão disponíveis gratuitamente para *download* em muitos formatos, e representam uma excelente oportunidade para praticar *Data Science* pois:
- Todo o conjunto de dados é gerado pelo usuário, significando que haverá uma quantidade significativa de dados "sujos";
- O conjunto de dados é para qualquer área é gratuito para baixar em muitos formatos, incluindo XML;
- Os dados são confiáveis e compreensíveis pelo ser humano porque representam lugares e recursos reais.

### 1.1 Objetivo do projeto

O objetivo deste projeto é obter dados do mapa de uma região do mundo; auditar os dados; corrigir os problemas encontrados; importar os dados em um banco de dados (nesse caso um banco de dados NoSQL, **MongoDB**) e executar algumas consultas exploratórias.

## 2. Coleta e Auditoria de Dados

### 2.1 Escolha do mapa

Decidi trabalhar com a área de Campinas, porque é onde eu atualmente moro. Os dados de Campinas usados para este projeto foram obtidos no OpenStreetMap e baixados seguinte link: https://www.openstreetmap.org/relation/298227

Algumas características desses dados são apresentadas abaixo:

<img src="map.png" />

### 2.2 Exploração preliminar dos dados

O primeiro passo foi baixar o mapa como um arquivo XML utilizando a API Overpass, conforme mostrado abaixo:

<img src="map2.png" />

Existem três elementos principais de nível superior no OSM, como pode-se verificar no código abaixo:
1. `'Nodes'` representam um único ponto e possuem id, latitude e longitude. Eles também podem conter *tags* descritivas no `'node'` se estiverem em um item de interesse;
2. `'Ways'` são constituídas por listas ordenadas de nós que descrevem uma característica linear, como uma trilha ou uma área como um parque. Eles contêm uma lista dos nós que compõem o caminho, bem como tags para informações detalhadas;
3. `'Relations'` são constituídas por uma lista ordenada de membros que podem ser `'nodes'` ou `'ways'`. Eles são usados para representar relações lógicas ou geográficas entre recursos e conter uma lista de membros, bem como *tags* que descrevem o elemento.

In [1]:
# Importação das bibliotecas necessárias para a análise

from xml.sax.handler import ContentHandler
from xml.sax import make_parser
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import unicodedata
import pycep_correios # Biblioteca dos Correios para consulta de CEP
import codecs
import json
from pymongo import MongoClient
from pymongo import GEO2D

In [10]:
''' Começando pelo começo: verificando se o arquivo OSMFILE é um arquivo XML válido. '''

file = "map.os" # É necessário descompactar o arquivo map.os.zip antes de iniciar a análise

def parse_file(file):
    """ Verifica se um determinado arquivo possui estrutura XML parseável.
    Args:
        file: arquivo a ser verificado.
    Raises:
        Exception: caso o arquivo não seja XML parseável.
    """
    osm_file = open(file, "r")
    parser = make_parser()
    parser.parse(osm_file)
    osm_file.close()
    
try:
    parse_file(file)
    print("%s is a XML file well-formed" % file)
except (Exception, e):
    print("%s is NOT a XML file well-formed! %s" % (file, e))

map.os is a XML file well-formed


In [11]:
''' Olhando dentro da estrutura do XML.'''

def process_file(file):
    """ Processa um arquivo XML e retorna quais são as tags únicas e suas contagens
    Args:
        file: arquivo a ser processado.
    Returns:
        Um conjunto de tags únicas encontradas no arquivo e suas respectivas contagens.
    """
    osm_file = open(file, "r")
    xml_tags = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag in xml_tags: # Se elemento já existe no conjunto, adiciona contagem
            xml_tags[elem.tag] += 1
        else: # Caso contrário, adiciona elemento e inicia com 1 contagem
            xml_tags[elem.tag] = 1
    osm_file.close()
    return xml_tags

result = process_file(file)
pprint.pprint(result)

{'bounds': 1,
 'member': 5430,
 'meta': 1,
 'nd': 343980,
 'node': 269600,
 'note': 1,
 'osm': 1,
 'relation': 467,
 'tag': 120097,
 'way': 42004}


In [12]:
''' Validando os atributos k do elemento tag, para ver se haverá algum problema ao carregar os dados no MongoDB'''

lower = re.compile(r'^([a-z]|_)*$') # Matches strings containing lower case characters
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') # Matches strings containing lower case characters and a single colon within the string
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') # Matches characters that cannot be used within keys in MongoDB

def key_type(element, keys):
    """ Processa os elementos XML e seus atributos k buscando por caracteres 
    problemáticos para o MongoDB.
    Args:
        elem: elemento XML.
        keys: conjunto de chaves.
    Returns:
        Um conjunto de tags únicas encontradas no arquivo e suas respectivas contagens.
    """
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys

def process_keys(file):
    osm_file = open(file, "r")
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(osm_file):
        keys = key_type(element, keys)
    osm_file.close()
    return keys

keys = process_keys(file)
pprint.pprint(keys)

{'lower': 114844, 'lower_colon': 5171, 'other': 82, 'problemchars': 0}


Como pode-se observar pelo resultado, nenhum problema foi encontrado no arquivo.

In [13]:
''' Identificando os usuários únicos'''

def process_users(file):
    """ Encontra o número de usuários únicos em um arquivo OSMFILE.
    Args:
        file: arquivo XML a ser processado.
    Returns:
        Número de usuários únicos.
    """
    osm_file = open(file, "r")
    users = set()
    for _, element in ET.iterparse(osm_file):
        for e in element:
            if 'user' in e.attrib:
                users.add(e.attrib['user'])
    osm_file.close()
    return users

users = process_users(file)
len(users)

537

No arquivo selecionado, foram encontrados 537 usuários únicos que contribuíram para os dados do OSMFILE.

In [6]:
''' Descobrindo quais são as tags únicas.'''

def process_tags(file, tag):
    """ Encontra as subtags únicas dentro de uma determinada tag.
    Args:
        file: arquivo XML a ser processado.
        tag: tag a ser verificada.
    Returns:
        Subtags únicas.
    """
    osm_file = open(file, "r")
    subtag_types = set() # Criando um set, pois gostaria de ver todas as subtags das tags.
    for event, element in ET.iterparse(osm_file, events=("start",)):
        if element.tag == tag:
            for subelem in element.iter():
                subtag_types.add(subelem.tag)
    osm_file.close()
    return subtag_types

for i in result:
    subtags = process_tags(file,i)
    print(i,subtags)

relation {'relation', 'tag', 'member'}
meta {'meta'}
note {'note'}
way {'nd', 'tag', 'way'}
osm {'note', 'osm', 'node', 'meta', 'tag', 'bounds'}
nd {'nd'}
member {'member'}
tag {'tag'}
bounds {'bounds'}
node {'tag', 'node'}


Nota-se que apenas as tags `relation`, `node`, `way` possuem uma estrutura com sub-elementos, assim como o próprio elemento *root* `osm`.

### 2.3 *Data Cleaning Process*
Estar no controle de um grande conjunto de dados pode ser excitante e intimidante! Existem inúmeras possibilidades de exploração, mas a grande quantidade de informações pode ser esmagadora. Antes de iniciar oficialmente qualquer análise de dados, é importante ter um plano para passar eficientemente pelo processo de auditoria e limpeza dos dados.A seguinte abordagem é bastante interessante, foi adaptada do curso Udacity sobre Data Wrangling:
1. Auditar os dados: identificar erros/dados em falta ou geralmente "sujos" no arquivo XML original;
2. Criar um plano de limpeza de dados com base na auditoria:
    - Identificar as causas de dados inconsistentes/incorretos;
    - Desenvolver um conjunto de ações de limpeza corretivas e teste em uma pequena amostra dos dados XML;
3. Implementar o plano de limpeza de dados: executando scripts de limpeza e transfira os dados limpos para arquivos .csv;
4. Corrijir manualmente conforme necessário: importando os dados dos arquivos .csv para um banco de dados NoSQL (ou, alternativamente, para um banco de dados SQL) e executando consultas nos dados para identificar quaisquer inconsistências adicionais que exijam o retorno ao passo 1.

A análise de dados é um procedimento iterativo e, como tal, é esperado trabalhar várias vezes com essas etapas. Além disso, é sempre bom ter um esboço claro do procedimento para não se perder no meio da análise.

### 2.3.1 Qualidade dos dados
Existem cinco aspectos principais da qualidade dos dados a serem considerados ao auditar um conjunto de dados:
1. **Validade**: os dados estão em conformidade com um formato padrão?
2. **Precisão**: os dados concordam com a realidade ou com uma fonte externa confiável? 
3. **Completude**: todos os registros estão presentes?
4. **Consistência**: os dados estão em um campo ou em uma linha em acordo lógico? 
5. **Uniformidade**: as mesmas unidades são usadas para um determinado campo?

### 2.3.2 Auditando e Limpando os Dados

#### Street types

Outra auditoria que eu poderia realizar para validade, bem como consistência, dizia respeito aos nomes das ruas associados ao nó e à tag. A partir do meu exame exploratório inicial dos dados, notei uma grande variedade de terminais e abreviaturas de nomes de ruas. Usando um script de auditoria de nome de rua provisório, eu comparei os terminais de rua com uma lista padronizada e contei o número de vezes que cada tipo não padrão apareceu usando a seguinte função

In [7]:
''' Auditando os tipos de ruas presentes na região escolhida.'''

street_type_re = re.compile(r'^\S+\.?(\b)?', re.IGNORECASE) # Regex para pegar a primeira palavra (https://regex101.com)

expected = ["Acesso", "Alameda", "Avenida", "Beco", "Boulevard", "Caminho",
    "Campo", u"Condomínio", "Estrada", "Ladeira", "Largo", "Parque", u"Praça",
    "Praia", "Rodovia", "Rua", "Travessa", "Via"]

mapping = { "Av": "Avenida",
            "Av.": "Avenida",
            "Est.": "Estrada",
            "Estr.": "Estrada",
            "estrada": "Estrada",
            "Pca": u"Praça",
            "Praca": u"Praça",
            u"Pça": u"Praça",
            u"Pça.": u"Praça",
            "R.": "Rua",
            "Rua.": "Rua",
            "RUA": "Rua",
            "rua": "Rua",
            "Ruas": "Rua",
            "Rue": "Rua",
            "Rod.": "Rodovia",
            "Trav": "Travessa" }

def update_street_name(name, mapping):
    """ Atualiza o nome da rua utilizando um dicionário.
    Args:
        name: nome da rua.
        mapping: dicionário.
    Returns:
        Nome corrigido.
    """
    m = street_type_re.search(name)
    street_type = m.group()
    if street_type in mapping:
        name = street_type_re.sub(mapping[street_type], name).title()
        name = name.replace("De", "de") # Corrigindo o "De" para "de" depois de ter aplicado title()
    return name

def audit_street_name(street_types, street_name):
    """ Audita um nome de rua.
    Args:
        street_types: dicionário com os tipos de ruas.
        street_name: nome da rua.
    Returns:
        Nome corrigido.
    """
    street_name = street_name.replace("Dr.", "Doutor") # Cleanup do Dr. para Doutor para uniformidade dos nomes.
    m = street_type_re.search(street_name) # Cria os grupos do dicionário
    if m:
        street_type = m.group()
        if street_type not in expected: # Adiciona ocorrência não esperada ao grupo
            street_types[street_type].add(street_name)
            street_name = update_street_name(street_name, mapping)
            print(street_name)
    return street_name

def audit_street(osmfile):
    """ Audita todos os nomes de ruas de um arquivo OSMFILE.
    Args:
        osmfile: arquivo XML do OpenStreetMap.
    Returns:
        Dicionário com todos os tipos de ruas.
    """
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:street":
                    audit_street_name(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

st_types = audit_street(file)
print("Streets in the wrong format:")
pprint.pprint(st_types)

Cecília Feres Zogbi
Avenida Doutor Armando de Ottaviano
Rua Bernardo de Souza Campos
Avenida Maria Ferreira Antunes
Doutor Bonifácio de Castro Filho
Doutor Bonifácio de Castro Filho
Doutor Bonifácio de Castro Filho
José Orides Cordeiro
das Castanheiras
Rua Laércio de Oliveira
Avenida James Clark Maxwell
Rua Mato Grosso 58 Monte Mor Sp
Avenida José Rocha Bomfim
das Pitangueiras
Avenida Mirandópolis
Avenida Doutor Moraes Sales
Avenida José Rocha Bomfim
Avenida José Rocha Bomfim
Plínio Pereira da Cruz
Campos Salles
Romeu Chiminasso
Avenida Doutor Armando de Ottaviano
Avenida Doutor Armando de Ottaviano
Avenida Francisco Sáles Píres
SP-332
Rua Vitor Baranauska Filho.N.125.
Rua Benedito Cândido Ramos
Rua Flambolant
Antônio Ferreira
Cecília Feres Zogbi
Avenida Cabo Pedro Hoffman
Avenida Cabo Pedro Hoffman
Rua Osvaldo Ribeiro Carrilho
Streets in the wrong format:
defaultdict(<class 'set'>,
            {'Antônio': {'Antônio Ferreira'},
             'Av': {'Av José Rocha Bomfim'},
             

**Descobertas**: aqui notamos que o uso de abreviações, como Av. e R. são as maiores ocorrências de tipo de rua que desviam do esperado. O uso de letras minúsculas ao invés de maiúsculas para o nome das ruas e avenidas também indica uma inconsistência, como a `rua bernardo de souza campos` e `Rua Bernardo de Souza Campos`, que representam a mesma rua. Além disso, existem nomes sem a determinação do tipo de rua (para esses últimos vamos deixar assim como está, mas que poderia ser corrigida uma a uma). 

**Correções**: As correções necesárias são sobre o tipo de rua e ruas que estão escritas em *lowercase*. As demais correções serão deixadas como trabalhos futuros.

#### Código Postal
Uma auditoria adicional que eu poderia realizar para verificar a precisão estava nos códigos postais. A minha abordagem com os códigos postais foi comparar os códigos postais encontrados com o serviço disponibilizado pelos Correios, bem como uniformizá-los. O código a seguir foi usado para executar a auditoria:

In [8]:
''' Auditando os tipos de CEPs presentes na região escolhida'''

postal_code_type_re = re.compile(r'^\d{2}[\.]?[0-9]{3}[\-\s]?[0-9]{3}', re.IGNORECASE) # Regex para pegar os CEPs no formato aceito no Brasil

def update_postal_code(code):
    """ Atualiza a formatação do CEP.
    Args:
        code: CEP.
    Returns:
        CEP corrigido.
    """
    for ch in ['.','-']:
        code=code.replace(ch,'') # Remove pontos e traços nos CEPs (coisa simples)
    return code

def audit_postal_code(osmfile):
    """ Audita todos códigos postais de um arquivo OSMFILE no formato brasileiro.
    Args:
        osmfile: arquivo XML do OpenStreetMap.
    Returns:
        Dicionário com todos os tipos de CEPs bons e um outro dicionário com todos
        os tipos de CEPs ruins.
    """
    osm_file = open(osmfile, "r")
    postal_codes = set()
    bad_zips = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == 'tag':
            if 'postal_code' in elem.attrib['k'] or 'addr:postcode' in elem.attrib['k']:
                zip_code = elem.attrib['v']
                test = postal_code_type_re.search(zip_code)
                if test:
                    postal_codes.add(zip_code)
                else:
                    bad_zips.add(zip_code)
    osm_file.close()
    return postal_codes, bad_zips

cep_types, cep_problematic = audit_postal_code(file)

print("Postal codes in the wrong format:")
pprint.pprint(cep_types)
print("Postal codes in the wrong format:")
pprint.pprint(cep_problematic)

# Verificando, por exemplo, se todos os CEPs são de Campinas
# Pode-se elaborar mais, verificando se o endereço das tags com CEP bate com o endereço registrado nos Correios
#for cep in cep_types:
    #endereco = pycep_correios.consultar_cep(cep)
    #assert endereco['cidade'] == 'Campinas'

Postal codes in the wrong format:
{'13.025-151',
 '13010-041',
 '13010-050',
 '13010-141',
 '13010-201',
 '13010-210',
 '13012-000',
 '13012-970',
 '13013-070',
 '13015-201',
 '13015-904',
 '13023-075',
 '13024-045',
 '13024-100',
 '13024-120',
 '13024-160',
 '13024-410',
 '13024-411',
 '13025-002',
 '13025-152',
 '13025-320',
 '13025050',
 '13025240',
 '13026-137',
 '13030-100',
 '13030-400',
 '13030-405',
 '13031-880',
 '13033-029',
 '13033-050',
 '13035-000',
 '13035-110',
 '13035-680',
 '13035270',
 '13036-100',
 '13036-225',
 '13036-902',
 '13040-565',
 '13040109',
 '13041-670',
 '13042-901',
 '13042105',
 '13043810',
 '13044290',
 '13045-603',
 '13046-260',
 '13046-355',
 '13046-370',
 '13046-380',
 '13046-540',
 '13049437',
 '13050-014',
 '13050-080',
 '13050-089',
 '13053-111',
 '13053-631',
 '13053-636',
 '13054-000',
 '13054-040',
 '13054-107',
 '13054-709',
 '13056-399',
 '13056-450',
 '13057-545',
 '13057400',
 '13057535',
 '13057541',
 '13058-533',
 '13058-581',
 '13060-19

**Descobertas**: Há poucas informações sobre o CEP, mas as informações encontradas batem com o esperado. Por exemplo, o CEP 13035-110 corresponde à `Rua Ernesto Segalho` e o CEP corresponde à `Rua Odila Santos de Souza Camargo`, ou seja, são CEPs existentes, são de Campinas, e as ruas dessas tags são as mesmas das esperadas. Para verificar em massa os CEPs em cidades do Brasil, recomenda-se o uso da biblioteca `pycep-correios` (https://pypi.python.org/pypi/pycep-correios/2.2.0) dos Correios, e que seja executado fora do laço de iteração de verificação das tags, pois a consulta aos Correios acaba tomando um tempinho que, dependendo do tamanho do arquivo OSMFILE, poderia levar muito tempo.

**Correções**: Exceto pelas três ocorrências, todos os CEPs atendem ao padrão Brasileiro, então nenhuma correção será necessária (como trabalho futuro, o que poderia ser feito é corrigir cada um dos códigos postais problemáticos usando como base as informações de endereço do respectivo elemento). No entanto, para deixar tudo certinho vamos eliminar os `"."` e `"-"` e uniformizar os CEPs. Como trabalho futuro, pode-se montar uma relação com todos os CEPs de Campinas e região e complementar o arquivo XML com essa informação.

#### Número de telefone

Em diversas aplicações, é importante padronizar o número de telefone. E com bastante frequência cada usuário coloca o telefone de um jeito. Assim, é possível automatizar ligações, envios de mensagens, SMS, organizar os contatos para a equipe de vendas, etc. É o que será feito abaixo, vamos ver quais os formatos de telefone no arquivo e vamos corrigir e padronizar para que os números de telefone contenham o formato +55 + número sem ou com o DDD (sem o zero). Essa parte é particularmente importante, especialmente pois varia muito de pessoa a pessoa como registrar o número de telefone e, principalemente, porque depois podemos usar essa informação de diversas formas.

In [9]:
''' Auditando os tipos de telefones presentes na região escolhida. Essa parte é particularmente importante,
especialmente pois varia muito de pessoa a pessoa como registrar o número de telefone e, principalemente,
porque depois podemos usar essa informação de diversas formas.'''

phone_type_re = re.compile(r'(?:[+]55)(?:([0-9]{2}))?(?:([0-9]{1}[0-9]{6,8}[0-9]))', re.IGNORECASE) # Regex para validar telefones no formato +55dddxxxxxxxxx ou +55xxxxxxxxx para ficar certo para uso em aplicações
number_re = re.compile(r'^\D*(\d+)', re.IGNORECASE) # Regex para pegar a primeira ocorrência de número em uma string

def update_phone_value(phone):
    """ Atualiza o número de telefone no padrão brasileiro.
    Args:
        phone: número de telefone.
    Returns:
        Telefone corrigido no padrão brasileiro.
    """
    for ch in [' ','-','(',')']: # Remove espaços, traços e parênteses
        phone=phone.replace(ch,'')
    m = number_re.search(phone) # Remove strngs se houver
    phone = m.group()
    if phone.startswith('0') and not phone.startswith('0800'): # Se começar com 0 e não for 0800, tira o zero
        phone = phone[len('0'):]
        phone = "+55"+ phone
    if phone.startswith('55'): # Se não começar com +55, adiciona +55
        phone = "+"+ phone
    if not phone.startswith('+55'):
        phone = "+55"+ phone
    return phone

def audit_phone_value(phone):
    """ Audita um número de telefone.
    Args:
        phone: número de telefone
    Returns:
        Número corrigido.
    """
    test = phone_type_re.search(phone)
    if not test:
        phone = update_phone_value(phone)
    return phone

def audit_phone(osmfile):
    """ Audita todos os telefones de um arquivo OSMFILE.
    Args:
        osmfile: arquivo XML do OpenStreetMap.
    Returns:
        Dicionário com todos os telefones bons e um outro dicionário com todos
        os telefones ruins.
    """
    osm_file = open(osmfile, "r")
    phones = set()
    bad_phones = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == 'tag':
            if 'phone' in elem.attrib['k']:
                phone = elem.attrib['v']
                test = phone_type_re.search(phone)
                if test:
                    phones.add(phone)
                else:
                    bad_phones.add(phone)
    osm_file.close()
    return phones, bad_phones

good_phones, bad_phones = audit_phone(file)

print("Phones in the correct format:")
pprint.pprint(good_phones)
print("Phones in the wrong format:")
pprint.pprint(bad_phones)

Phones in the correct format:
{'+551932015700',
 '+551932544172',
 '+551932895930',
 '+551933251014',
 '+551933259500',
 '+551933678600',
 '+551933950335',
 '+551933967289',
 '+551937217710',
 '+551937294477',
 '+551937562700',
 '+5519988928053'}
Phones in the wrong format:
{'(19) 2103-9166',
 '(19) 2137 6803',
 '(19) 2137-0600',
 '(19) 2512-0304',
 '(19) 3202 5700',
 '(19) 3203 0101',
 '(19) 3231 2022',
 '(19) 3231 2121',
 '(19) 3231 4290',
 '(19) 3231-5436',
 '(19) 3233 7085',
 '(19) 3236 1222',
 '(19) 3236 9800',
 '(19) 3253-7364',
 '(19) 3256-8689',
 '(19) 3256-8799',
 '(19) 3277-0064',
 '(19) 3294 3892',
 '(19) 3296-5412',
 '(19) 3327-2574',
 '(19) 3519 3829',
 '(19) 3731 2430',
 '(19) 3734 3000',
 '(19) 3736 9500',
 '(19) 3737-4390',
 '(19) 3739 3004',
 '(19) 3739 8888',
 '(19) 3753-2400',
 '(19) 3755-8000',
 '(19) 3773 9000',
 '(19) 3829-5120',
 '(19) 3845 2015',
 '(19) 3845 2672',
 '(19) 3869-3900',
 '(19) 3897 1900',
 '(19) 98128-8127',
 '+55  19 3844 8532',
 '+55 (19) 2121-19

**Descobertas**: Aqui realmente encontrou-se vários formatos de números: com espaço, com traço, com DDD entre parênteses sem o código nacional +55, e até em uma frase `'4090-1030 para capitais e regiões metropolitanas e 0800-883-2000 para demais '`. Esse é um campo, portanto, que devemos dar atenção ao passar para o MongoDB.

**Correções**: A correção consistiu em formatar todos os números ruins para +55 sem ou com o DDD (sem o zero).

#### Formato do website

Novamente, aqui é importante verificar se os atributos `'websites'` estão corretos. Isso também é importante, porque pode ser necesário acessar o link presente nesses campos para fazer alguma coisa, como um screen scraping para
obter dados adicionais de alguma tag (como um restaurante ou hospital).

In [10]:
''' Auditando os tipos de URL presentes nos campos 'source' e 'website'. Isso também é importante, porque pode 
ser necesário acessar o link presente nesses campos para fazer alguma coisa, como um screen scraping para
obter dados adicionais de alguma tag (como um restaurante ou hospital).'''

website_type_re = re.compile(r'^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$', re.IGNORECASE) #Regex para websit

def audit_url(osmfile):
    """ Audita todas as URLs um arquivo OSMFILE.
    Args:
        osmfile: arquivo XML do OpenStreetMap.
    Returns:
        Dicionário com todas as URLs boas e um outro dicionário com todas
        as URLs ruins.
    """
    osm_file = open(osmfile, "r")
    websites = set()
    bad_websites = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == 'tag':
            if 'website' in elem.attrib['k']:
                site = elem.attrib['v']
                test = website_type_re.search(site)
                if test:
                    websites.add(site)
                else:
                    bad_websites.add(site)
    
    osm_file.close()
    return websites, bad_websites

good_urls, bad_urls = audit_url(file)

print("URLs in the correct format:")
pprint.pprint(good_urls)
print("URLs in the wrong format:")
pprint.pprint(bad_urls)

URLs in the correct format:
{'educacao.sp.gov.br',
 'http://acainow.com.br/',
 'http://arquidiocesecampinas.com/local/paroquia-imaculado-coracao-de-maria',
 'http://arquidiocesecampinas.com/local/paroquia-santo-expedito',
 'http://assumasustentabilidade.org.br',
 'http://bardocarioca.com.br/',
 'http://bardomarcelino.com.br/',
 'http://benavitta.com.br/',
 'http://cafeearte.com.br/',
 'http://ceak.org.br/',
 'http://charliespub.com.br',
 'http://chrissports.com.br/',
 'http://cmp.ifsp.edu.br',
 'http://derosecampinas.com.br/',
 'http://dgrh.unicamp.br/',
 'http://espaco.mulheresempoderadas.com.br/',
 'http://estacaomarupiara.com.br/',
 'http://estanciadoliveirabarao.com.br/',
 'http://fb.com/TalentFightGym',
 'http://gatomiacafe.com.br/',
 'http://golagelato.com.br/',
 'http://goodbom.com.br/',
 'http://hospitaldocoracao.com',
 'http://kitmix.com.br/',
 'http://lhc.net.br',
 'http://linapenteado.blogspot.com.br/',
 'http://maaxveiculos.com.br/',
 'http://mackenzie.br',
 'http://nautas.

**Descobertas**: Não foi encontrada nenhuma URL em algum formato estranho. Alguns endereços faltam o `'http://'` ou `'https://'`, outros o `'www'`, mas nada que interfira no acesso ao website.

**Correções**: Nenhuma correção necessária.

#### Formato do e-mail

Por fim, aqui é importante verificar se os atributos `'email'` estão corretos. Isso é importante, porque pode ser necesário enviar  um e-mail para o valor presente nesses campos para fazer alguma coisa (como uma notificação de alteração por exemplo).

In [11]:
''' Auditando os tipos de e-mails presentes nos campos 'email'. Isso é importante, porque pode ser necesário enviar 
um e-mail para o valor presente nesses campos para fazer alguma coisa (como uma notificação de alteração por exemplo).'''

email_type_re = re.compile(r'^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$', re.IGNORECASE) # Regex para validar email

def audit_email_value(email):
    """ Atualiza o endereço de email.
    Args:
        email: edereço de email.
    Returns:
        Email corrigido.
    """
    nfkd = unicodedata.normalize('NFKD', email) # Unicode normalize transforma um caracter em seu equivalente em latin.
    email = u"".join([c for c in nfkd if not unicodedata.combining(c)])   
    return email

def audit_email(osmfile):
    """ Audita todos os emails de um arquivo OSMFILE.
    Args:
        osmfile: arquivo XML do OpenStreetMap.
    Returns:
        Dicionário com os emails bons e um outro dicionário com todos
        os emails ruins.
    """
    osm_file = open(osmfile, "r")
    emails = set()
    bad_emails = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == 'tag':
            if 'email' in elem.attrib['k']:
                email = elem.attrib['v']
                test = email_type_re.search(email)
                if test:
                    emails.add(email)
                else:
                    if website_type_re.search(email):
                        emails.add(email)
                    else:
                        bad_emails.add(email)
    osm_file.close()
    return emails, bad_emails

good_emails, bad_emails = audit_email(file)

print("E-mails in the correct format:")
pprint.pprint(good_emails)
print("E-mails in the wrong format:")
pprint.pprint(bad_emails)

E-mails in the correct format:
{'7gb@polmil.sp.gov.br',
 'aforcafitness@gmail.com',
 'atendimento@vidrocarcamp.com.br',
 'audiotecatpln@gmail.com',
 'balcaoir@vidrocarcamp.com.br',
 'cassiajuliato@hotmail.com',
 'cdicampinas@cdicampinas.org.br',
 'cel@cel.unicamp.br',
 'contact@profusion.mobi',
 'contato@pastificioselmi.com.br',
 'contato@veraqueiroz.com.br',
 'cpphorto@sap.sp.gov.br',
 'dtd@cdpho.sap.sp.gov.br',
 'e017255a@educacao.sp.gov.br',
 'e018892a@educacao.sp.gov.br',
 'e036468a@educacao.sp.gov.br',
 'e039858a@educacao.sp.gov.br',
 'e039861a@educacao.sp.gov.br',
 'e042523a@educacao.sp.gov.br',
 'e045559a@educacao.sp.gov.br',
 'e045697a@educacao.sp.gov.br',
 'e047806a@educacao.sp.gov.br',
 'e048240a@educacao.sp.gov.br',
 'e048264a@educacao.sp.gov.br',
 'e048276a@educacao.sp.gov.br',
 'e070294a@educacao.sp.gov.br',
 'e462019a@educacao.sp.gov.br',
 'e907352a@educacao.sp.gov.br',
 'e907595a@educacao.sp.gov.br',
 'e909610a@educacao.sp.gov.br',
 'e917552a@educacao.sp.gov.br',
 'e9179

**Descobertas**: Os únicos erros encontrados foram: um acento em um e-mail e um link para o formulário de contato. Em caso de link para formulário de contato, vamos manter, caso queira se realizar um screen scraping para enviar e-mail ou repassar o link para alguém acessar e enviar a mensagem.

**Correções**: Como os erros foram poucos e simples, nesse caso é necessário apenas eliminar acentos (já que esse usuário pode ter colocado em outras ocorrências ao longo de outras partes do OSM para arquivos de outras regiões).

### 2.3.3 Problemas encontrados

Os principais problemas encontrados foram nos tipos de ruas e também em alguns nomes, também foram encontrados diversos erros na formatação de telefone e alguns erros em e-mails. As rotinas de correção (*cleanup*) foram implementadas acima, juntamente com as de auditoria dos dados.

Agora, portanto, é possível carregar as informações no MongoDB e realizar consultas para descobrir números interessantes da região escolhida. A ideia é focar em pesquisas úteis, como por exemplo, quantidade de restaurantes na região e suas características (para uso interno em aplicações como iFood por exemplo), ou a quantidade de hospitais, heliportos e outros estabelecimentos de assistência e suas características (para uso interno em aplicações como Youse).

## 3. Data Wrangling com MongoDB

### 3.1 Criando o arquivo .json auditado e limpo

In [12]:
''' O primeiro passo aqui é, portanto, transformar o arquivo .osm auditado e limpo em um arquivo .json'''

CREATED = ["version", "changeset", "timestamp", "user", "uid"]

def handle_email(v):
    """ Formata email.
    Args:
        v: email não formatado.
    Returns:
        Retorna email formatado.
    """
    if email_type_re.search(v):
        return v
    elif website_type_re.search(v):
        return v
    else:
        return audit_email_value(v)

def handle_address(node, parts, v):
    """ Formata endereço.
    Args:
        node: nó.
        parts: split do valor do atributo.
        v: endereço não formatado.
    Returns:
        Retorna nó sem nenhuma alteração caso número de partes do split do 
        atributo seja maior que 2.
    """
    v = v.replace("Dr.", "Doutor") # Cleanup do Dr. para Doutor para uniformidade dos nomes.
    if len(parts) > 2:
        return node
    if parts[1] == 'street':
        v = update_street_name(v, mapping)
    if 'address' not in node:
        node['address'] = dict()
    node['address'][parts[1]] = v

def handle_phone(phone):
    """ Formata número do telefone.
    Args:
        phone: número do telefone não formatado.
    Returns:
        Retorna número do telefone formatado.
    """
    phones = re.split(';|,', phone) # separa em múltiplos telefones
    updated_phones = list()
    for p in phones:
        if len(p) == 0: # drop strings vazias
            continue
        p = audit_phone_value(p)
        updated_phones.append(p)
    return updated_phones

def shape_attribute(node, k, v):
    """ Formata atributos de um determinado nó dada sua chave k e o valor dela.
    Args:
        node: nó.
        k: chave.
        v: valor da chave.
    Returns:
        Retorna um elemento node formatado.
    """
    if k in CREATED:
        if 'created' not in node:
            node['created'] = dict()
        node['created'][k] = v
    elif (k == 'lat') or (k == 'lon'):
        if 'pos' not in node:
            node['pos'] = list()
        if k == 'lon':
            node['pos'].insert(0, float(v))                
        elif k == 'lat':
            node['pos'].append(float(v))
    else:
        node[k] = v
    return node

def shape_nd(node, tag):
    """ Formata os elementos node_refs de um determinado.
    Args:
        node: nó.
        tag: tag.
    Returns:
        Retorna um elemento node formatado.
    """
    if 'node_refs' not in node:
        node['node_refs'] = list()
    node['node_refs'].append(tag.attrib.get('ref'))
    return node

def shape_tag(node, tag):
    """ Formata uma determinada tag e seus atributos.
    Args:
        node: nó.
        tag: tag.
    Returns:
        Retorna um elemento node formatado.
    """
    k = tag.attrib.get('k')
    v = tag.attrib.get('v')
    if problemchars.search(k):
        return node
    elif ':' in k:
        parts = k.split(':')
        if parts[0] == 'addr':
            node = handle_address(node, parts, v)
        elif parts[1] == 'phone' or parts[1] == 'fax':
            k = '_'.join(parts)
            node[k] = handle_phone(v)
        else:
            k = '_'.join(parts)
            node[k] = v
    else:
        if k == 'type':
            k = 'type_tag'
        if k == 'phone':
            v = handle_phone(v)
        elif k == 'postal_code':
            v = update_postal_code(v)
        elif k == 'email':
            v = handle_email(v)
        node[k] = v
    return node

def handle_helipad_elevation(node):
    """ Formata um elemento helipad.
    Args:
        node: nó.
    Returns:
        Retorna um elemento helipad formatado.
    """
    if ('aeroway' in node) and (node['aeroway'] == 'helipad'):
        if 'ele' in node: # 'ele' significa elevation
            try:
                node['ele'] = int(node['ele']) # Converte para int a altitude registrada para o heliporto
            except ValueError:
                node.pop('ele') # Se não for possível, remove

def shape_element(element):
    """ Formata um elemento XML OSMFILE para um elemento JSON.
    Args:
        element: elemento XML.
    Returns:
        Retorna um elemento JSON.
    """
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        for key, value in element.attrib.items():
            shape_attribute(node, key, value)
        for tag in element.iter('nd'):
            shape_nd(node, tag)
        for tag in element.iter('tag'):
            shape_tag(node, tag)
        handle_helipad_elevation(node)
        return node
    else:
        return None

def process_map(file_in, pretty=False):
    """ Lê um aquivo XML OSMFILE e o transforma em um arquivo JSON.
    Args:
        file_in: arquivo OSMFILE a ser processado.
        pretty: se True, formata visualmente o arquivo JSON resultante,
            se False apenas pula de linha a cada registro.
    Returns:
        Retorna os elementos XML do OSMFILE convertidos em elementos JSON.
    """
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "wb", 'utf-8') as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2) + "\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

data = process_map('map.os', True)

### 3.2 Importando os documentos para o MongoDB

Para importar os documentos para o MongoDB, deve-se realizar o seguinte comando:

> brew services start mongodb

> mongoimport --db project --collection open_street_map --drop --file map.os.json

Para acessar, portanto, o Banco de Dados:

In [2]:
client = MongoClient("mongodb://localhost:27017") # Acessando a db 'project'
db = client['project']

### 3.3 Realizando consultas básicas para verificar a consistência

In [3]:
db.open_street_map.find().count() # contagem do número de documentos

311604

In [4]:
db.open_street_map.find({'type': 'node'}).count() # contagem das tags node

269600

In [6]:
db.open_street_map.find({'type': 'way'}).count() # contagem das tags way

42004

In [8]:
def aggregate(db, pipeline):
    """ Agrega os resultados de uma consulta ao banco de dados Mongo.
    Args:
        db: banco de dados a ser pesquisado.
        pipeline: pipeline de consulta.
    Returns:
        Retorna resultado da consulta.
    """
    return [doc for doc in db.open_street_map.aggregate(pipeline)]

distinct_users = [
    {'$group': {'_id': '$created.user'}},
    {'$group': {'_id': 'Distinct users:', 'count': {'$sum': 1}}}]
result = aggregate(db, distinct_users)
pprint.pprint(result)

[{'_id': 'Distinct users:', 'count': 531}]


Se observar, o número de usuários foi menor que o que encontramos no XML do OSMFILE. Vamos investigar então.

In [15]:
the162_users = [
    {'$group': {'_id': '$created.user', 'count': {'$sum': 1}}},
    {'$sort': {'_id': 1}},
    {'$limit': 531}]
result = aggregate(db, the162_users)

In [16]:
# Vamos descobrir quem está no arquivo do XML e que não está no MongoDB (arquivo JSON)
mongo_users = []

for user in result:
    mongo_users.append(user['_id'])
    
for xml_user in users:
    i = 0
    if xml_user not in mongo_users:
        i = 1
    if i == 1:
        print(xml_user)

F cajuru
Denis Rosa
portalaventura
elkueb
Junior Intervales
joao pedro2019


Esses são os usuários que não estão no MongoDB mas que estão no OSMFILE. Se consultarmos o arquivo XML reparamos que são os usuários que contribuíram criando tags do tipo `'relation'`, as quais, como podemos ver no comando abaixo, decidimos por não carregar no MongoDB. Ou seja, a nossa base de dados indica estar consistente para avançarmos em análises mais interessantes.

In [17]:
db.open_street_map.find({'type': 'relation'}).count() # contagem das tags relation

0

### 3.3 A parte interessante: consultas aos dados

Abaixo, é possível visualizar os 10 usuários que mais contribuiram editando o mapa usado nesse projeto.

In [18]:
top_10_users = [
    {'$group': {'_id': '$created.user', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 10}]
result = aggregate(db, top_10_users)
pprint.pprint(result)

[{'_id': 'patodiez', 'count': 44170},
 {'_id': 'Gustavo Alves', 'count': 27247},
 {'_id': '~AR33~', 'count': 26699},
 {'_id': 'AjBelnuovo', 'count': 17733},
 {'_id': 'Daniel Assuncao', 'count': 15705},
 {'_id': 'igorkalju', 'count': 15120},
 {'_id': 'MCPicoli', 'count': 13899},
 {'_id': 'PsyLu', 'count': 12942},
 {'_id': 'Thundercel', 'count': 8108},
 {'_id': 'kurka', 'count': 6911}]


Aqui, são contabilizados a quantidade de usuários que contribuiu apenas uma vez para o mapa desse projeto.

In [19]:
users_appearing_once = [
    {'$group': {'_id': '$created.user', 'count': {'$sum':1}}},
    {'$group': {'_id': '$count', 'num_users': {'$sum':1}}},
    {'$sort': {'_id': 1}},
    {'$limit': 1}]
result = aggregate(db, users_appearing_once)
pprint.pprint(result)

[{'_id': 1, 'num_users': 112}]


Quais as 10 `'amenities'` mais encontradas? É possível observar que a `'amenity'` restaurante (e também tem *fast food*), o que é interessante do ponto de vista de várias aplicações (como iFood, comentado anteriormente). 

In [20]:
most_common_amenities = [
    {'$match': {'amenity': {'$exists': 1}}},
    {'$group': {'_id': '$amenity', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 10}]
result = aggregate(db, most_common_amenities)
pprint.pprint(result)

[{'_id': 'restaurant', 'count': 270},
 {'_id': 'parking', 'count': 264},
 {'_id': 'school', 'count': 186},
 {'_id': 'fuel', 'count': 161},
 {'_id': 'place_of_worship', 'count': 155},
 {'_id': 'bank', 'count': 137},
 {'_id': 'telephone', 'count': 100},
 {'_id': 'fast_food', 'count': 90},
 {'_id': 'pharmacy', 'count': 87},
 {'_id': 'clinic', 'count': 55}]


E quais os tipos de cozinhas (`'cousines'`) mais comuns? Nessa consulta é possível verificar que cozinha reginal e pizzarias lideram, seguidas por culinária japonesa, churrascaria e culinária italiana.

In [21]:
top_10_cuisines = [
    {'$match': {'amenity': 'restaurant', 'cuisine': {'$exists': 1}}},
    {'$group': {'_id': '$cuisine', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 10}]
result = aggregate(db, top_10_cuisines)
pprint.pprint(result)

[{'_id': 'regional', 'count': 35},
 {'_id': 'pizza', 'count': 30},
 {'_id': 'japanese', 'count': 15},
 {'_id': 'barbecue', 'count': 9},
 {'_id': 'italian', 'count': 8},
 {'_id': 'burger', 'count': 7},
 {'_id': 'international', 'count': 6},
 {'_id': 'chinese', 'count': 6},
 {'_id': 'steak_house', 'count': 5},
 {'_id': 'ice_cream', 'count': 4}]


Aqui são verificados quais restaurantes possuem um telefone de contato registrado no banco de dados. Incrivelmente esse número é bastante pequeno.

In [22]:
top_10_streets = [
    {'$match': {'amenity': 'restaurant', 'address.street': {'$exists': 1}}},
    {'$group': {'_id': '$address.street', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 10}]
result = aggregate(db, top_10_streets)
pprint.pprint(result)

[{'_id': 'Rua Coronel Quirino', 'count': 6},
 {'_id': 'Rua dos Bandeirantes', 'count': 4},
 {'_id': 'Avenida Coronel Silva Teles', 'count': 4},
 {'_id': 'Rua Belo Horizonte', 'count': 3},
 {'_id': 'Avenida Júlio de Mesquita', 'count': 3},
 {'_id': 'Avenida Albino José Barbosa de Oliveira', 'count': 3},
 {'_id': 'Rua Barreto Leme', 'count': 2},
 {'_id': 'Rua João Mendes Júnior', 'count': 2},
 {'_id': 'Avenida Independência', 'count': 2},
 {'_id': 'Rua Gustavo Enge', 'count': 2}]


Nessa consulta, verificamos quais são as ruas com maior número de restaurantes.

In [23]:
restaurant_with_phone = [
    {'$match': {'amenity': 'restaurant', 'phone': {'$exists': 1}}},
    {'$group': {
            '_id': 'Restaurant with phone stats:',
            'count': {'$sum': 1}}}]
result = aggregate(db, restaurant_with_phone)
pprint.pprint(result)

[{'_id': 'Restaurant with phone stats:', 'count': 22}]


Por exemplo, se quiséssemos passar os telefones desses restaurantes, para uma equipe entrar em contato, bastaria executar o comando abaixo. E, se quiséssemos passar as localizações dos outros restaurantes bastava fazer a consulta alterando para `'{'$match': {'amenity': 'restaurant', 'phone': {'$exists': 1}}}'`.

In [24]:
restaurant_with_phone = [
    {'$match': {'amenity': 'restaurant', 'phone': {'$exists': 1}}}]
result = aggregate(db, restaurant_with_phone)
pprint.pprint(result)

[{'_id': ObjectId('5aa28ce5d51268e111d2409a'),
  'address': {'city': 'Campinas',
              'housenumber': '584',
              'postcode': '13084-008',
              'street': 'Avenida Albino José Barbosa de Oliveira',
              'suburb': 'Barao Geraldo'},
  'amenity': 'restaurant',
  'created': {'changeset': '52805191',
              'timestamp': '2017-10-10T20:10:45Z',
              'uid': '4176326',
              'user': 'mutuka',
              'version': '2'},
  'cuisine': 'italian',
  'id': '1143282936',
  'name': "Estância D'Oliveira",
  'phone': ['+551932895369', '+5532491510'],
  'pos': [-47.0787173, -22.8333069],
  'type': 'node',
  'website': 'http://estanciadoliveirabarao.com.br/'},
 {'_id': ObjectId('5aa28ce6d51268e111d2d9b9'),
  'address': {'housenumber': '39', 'street': 'Rua Belo Horizonte'},
  'amenity': 'restaurant',
  'created': {'changeset': '11018006',
              'timestamp': '2012-03-18T12:15:08Z',
              'uid': '397850',
              'user': 'Tul

Assim como o número de restaurantes com contato de e-mail no banco de dados:

In [25]:
restaurant_with_email = [
    {'$match': {'amenity': 'restaurant', 'email': {'$exists': 1}}},
    {'$group': {
            '_id': 'Restaurant with e-mail stats:',
            'count': {'$sum': 1}}}]
result = aggregate(db, restaurant_with_email)
pprint.pprint(result)

[{'_id': 'Restaurant with e-mail stats:', 'count': 1}]


E do número de restaurantes com website no banco de dados:

In [26]:
restaurant_with_website = [
    {'$match': {'amenity': 'restaurant', 'website': {'$exists': 1}}},
    {'$group': {
            '_id': 'Restaurant with website stats:',
            'count': {'$sum': 1}}}]
result = aggregate(db, restaurant_with_website)
pprint.pprint(result)

[{'_id': 'Restaurant with website stats:', 'count': 30}]


Por curiosidade, a consulta abaixo verifica quais desses restaurantes possuem área para fumante.

In [27]:
smoking = [
    {'$match': {'amenity': 'restaurant', 'smoking': {'$exists': 1}}},
    {'$group': {'_id': '$smoking', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}]
result = aggregate(db, smoking)
pprint.pprint(result)

[{'_id': 'no', 'count': 20}, {'_id': 'outside', 'count': 3}]


Por fim, vamos ver as principais lojas também na região. É possível observar  que há bastante padarias, supermercados e lojas de conveniência, o que também tem relação com possíveis aplicações ligadas ao consumo de alimentos.

In [28]:
top_10_shop = [
    {'$group': {'_id': '$shop', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 10}]
result = aggregate(db, top_10_shop)
pprint.pprint(result)

[{'_id': None, 'count': 310755},
 {'_id': 'supermarket', 'count': 132},
 {'_id': 'bakery', 'count': 81},
 {'_id': 'clothes', 'count': 49},
 {'_id': 'car', 'count': 40},
 {'_id': 'mall', 'count': 32},
 {'_id': 'car_repair', 'count': 31},
 {'_id': 'newsagent', 'count': 27},
 {'_id': 'hairdresser', 'count': 24},
 {'_id': 'convenience', 'count': 24}]


### 3.4 Aplicação divertida: geolocalização

Uma última coisa que podemos fazer, nas verdade uma das coisas mais interessantes que podemos fazer, é utilizar as informações e índices de geolocalização para agregar valor à nossa análise ou aplicação.

Campinas não é conhecida por possuir grande atrações turísticas, mas quase todos que visitam ou moram na cidade, acabam ouvindo falar da `'Torre do Castelo'`. 

Digamos que, em uma visita de amigos eu decida mostrá-la à eles. No entanto, após levá-los até a `'Torre do Castelo'`, como está na hora do almoço, gostaria de saber uma lista de restaurantes próximos. Para isso, poderia realizar a seguinte consulta em minha aplicação para me apresentar os restaurantes mais próximos de onde eu estou. 

In [15]:
db.open_street_map.create_index([('pos', GEO2D)])

my_position = db.open_street_map.find_one({'name': 'Torre do Castelo'})

result = db.open_street_map.find(
    {'pos': {'$near': my_position['pos']}, 'amenity': 'restaurant'},
    {'_id': 0, 'name': 1, 'cuisine': 1, 'address.street': 1}).skip(0).limit(10)

pprint.pprint([item for item in result])

[{'cuisine': 'barbecue', 'name': 'Churrascaria Chimarrão'},
 {'cuisine': 'burger', 'name': 'Big Jack'},
 {'cuisine': 'japanese', 'name': 'Restaurante Sakae'},
 {'cuisine': 'pizza', 'name': 'Serata Pizza Bar'},
 {'cuisine': 'pizza', 'name': 'Villa di Siena'},
 {'name': 'Panela de Barro'},
 {'cuisine': 'chicken', 'name': 'Vila del Gali'},
 {'cuisine': 'pizza', 'name': 'Vila Toscana Pizza e Bar'},
 {'cuisine': 'ice_cream', 'name': 'Sabor e Sonho'},
 {'cuisine': 'regional', 'name': "Casa d'Avó"}]


Quero também selecionar apenas os restaurantes 100m próximos ao meu local.

In [20]:
top_100m_near = [
    { "$geoNear": {
        "near": my_position['pos'],
        "maxDistance": 0.1/6378, # 0.1 km (ou 100m) mais próximos
        "spherical": "true",
        "distanceField": "distance",
        "distanceMultiplier": 6378, # diâmetro da Terra em km
        "query": {'amenity': 'restaurant'}
    }}]
result = aggregate(db, top_100m_near)
pprint.pprint(result)

[{'_id': ObjectId('5aa28ce6d51268e111d2b8bd'),
  'amenity': 'restaurant',
  'created': {'changeset': '13700944',
              'timestamp': '2012-10-31T15:37:45Z',
              'uid': '99811',
              'user': 'Camponez',
              'version': '2'},
  'cuisine': 'barbecue',
  'distance': 0.04934416504020776,
  'id': '1588346110',
  'name': 'Churrascaria Chimarrão',
  'pos': [-47.0763283, -22.8901036],
  'type': 'node'},
 {'_id': ObjectId('5aa28ce6d51268e111d2c931'),
  'amenity': 'restaurant',
  'created': {'changeset': '13700944',
              'timestamp': '2012-10-31T15:38:00Z',
              'uid': '99811',
              'user': 'Camponez',
              'version': '2'},
  'cuisine': 'burger',
  'distance': 0.052290421274735405,
  'id': '1605858810',
  'name': 'Big Jack',
  'pos': [-47.0763356, -22.8898775],
  'type': 'node'}]


Tanto a Churrascaria Chimarrão (49m de distância) quanto o Big Jack (52m de distância) parecem boas pedidas! :D

## 4. Reflexão

### 4.1 Completude e precisão.

O maior problema encontrado aqui é a falta de completude dos dados e de precisão. E no caso dessa aplicação, quanto maior completude alcançarmos, maior será o problema de precisão, pois informações de contato, bem como o nome do próprio estabelecimento, podem mudar com alguma frequência. Mas a análise ganharia bastante quanto mais completo e preciso fossem os dados do OSMFILE.

#### Solução: criação de um mecanismo de gamificação para incentivar a contribuição dos usuários

- Benefícios:   
    - Um sistema de gamification, estilo Waze, poderia contribuir muito para o aumento da completude e precisão das informações.
    - Também manteria o sistema bastante atualizado.


- Problemas:     
    - Encontrar um sistema de gamification que funciona é complicado, pode dar certo de primeira como pode ser um desatre completo (por isso Agile é altamente recomendado aqui).
    - Teria que conhecer melhor os usuários do OSM. Se o perfil deles não for o de pessoas que aceitam gamification, pode acabar dando errado.
    
#### Solução: integração com APIs externas para imputação de dados ausentes

- Benefícios:   
    - É uma forma bastante rápida de resolver o problema de completude e precisão.
    - Por exemplo, usar a própria API do próprio Google Maps.


- Problemas:     
    - O Google cobra pelo acesso à API do Maps dependendo do volume de requisições.
    - Cada solução tem suas peculiaridades, e no longo prazo manter essa base de código de integração com terceiros poderia ser um grande problema.


### 4.2 Open Street Map poderia adicionar validador de telefone, código postal, e-mail, e website

Com uma ação simples, como oferecer no frontend (e no backend) do Open Street Map funcionalidades de validação dos inputs de telefone, código postal, e-mail, e website pelos usuários contribuiria bastante em manter os dados consistentes e, principalmente, uniformes.

#### Solução: validação de campos

- Benefícios:   
    - Melhor solução, criar validadores para os principais campos.


- Problemas:     
    - Pode inibir o uso do sistema pelos usuários se não for bem implementado.

#### Solução: validação do código postal com o WebService dos Correios

- Benefícios:   
    - Todos os códigos postais em território nacional estariam no mesmo formato.


- Problemas:     
    - Se pensar do ponto de vista do OSM não seria a melhor solução ainda, pois funcionaria no Brasil. Mas e nos demais países? E integrar com a API de código postal de cada país (onde houver) vai certamente ser um ponto desafiador a longo prazo para manutenção de código.

## 5. Conclusão

Com pode-se perceber, e que já era previsto, os dados inseridos por pessoas quase sempre terão inconsistências e problemas de completude. E mesmo que uma grande parte dela seja inserida por bots, diferentes bots podem inserir dados usando padrões diferentes e a inconsistência permanece. Por outro lado, essa liberdade na entrada de dados confere muita flexibilidade aos usuários e, por isso, a representação do mapa pode ser ainda mais fiel ao mundo real.

A limpeza foi bastante útil para análises posteriores, validando a geolocalização dos elementos, bem como validando e formatando os campos de contatos e endereço.

## Referências

> Data cleansing, https://en.wikipedia.org/wiki/Data_cleansing

> Google Python Style Guide, https://google.github.io/styleguide/pyguide.html

> Google Python Style Guide (Comments Section), https://google.github.io/styleguide/pyguide.html?showone=Comments#Comments

> GeoNear, https://docs.mongodb.com/manual/reference/command/geoNear/#examples
